using Gadfly

# Distancias para las que se "agrega" una partícula a las interacciones
distances = Vector() 
append!(distances, sqrt(l1^2 + 1) + 0.01)
append!(distances, sqrt(l2^2 + 1) + 0.01)
append!(distances, sqrt((1 + l1)^2 + l2^2) + 0.01)
append!(distances, sqrt((1 + l2)^2 + l1^2) + 0.01)
append!(distances, sqrt(l1^2 + (1 - l2)^2) + 0.01)
append!(distances, sqrt(l2^2 + (1 - l1)^2) + 0.01)
append!(distances, 1 + l1 + 0.01)
append!(distances, 1 + l2 + 0.01)
append!(distances, 1 - l1 + 0.01)
append!(distances, 1 - l2 + 0.01)
append!(distances, 1 + 0.01)
sort(distances)

In [2]:
cell = [[0, 0], [0, l1], [l2, 0], [l2, l1]]
a = []
for jj in -1:1
    for ii in -1:1
        for c in cell
            if ii == 0 && jj == 0
                continue
            end
            append!(a, [c + [ii, jj]])
        end
    end
end
a

LoadError: UndefVarError: l1 not defined

In [31]:
dists = sort(unique(map(norm, a)))

20-element Array{Any,1}:
 0.1     
 0.460977
 0.55    
 0.559017
 1.0     
 1.00499 
 1.05475 
 1.09659 
 1.14127 
 1.34536 
 1.41421 
 1.45    
 1.45344 
 1.7066  
 1.76139 
 1.9     
 1.95256 
 1.978   
 2.14709 
 2.39008 

In [ ]:
diff = min([dists[ii] - dists[ii - 1] for ii in 2:length(dists)]...)

In [1]:
# Que la primera mitad sean derechas y la otra, izquierdas
const atoms_per_molecule = 3
const side = 12 # Número de celdas por lado de la caja de simulación, que debe ser un número par
const total_atoms = floor(Int64, side*side*atoms_per_molecule)

#global l1, n1, radius, n2, l2
const lambda = 1.001 # Rango de interacción
const l1 = 0.9 # Longitud del lado largo
const l2 = l1/2 # Longitud del lado corto
if lambda <= 1.005
    const nps = 1 # Columnas o filas al lado
else
    const nps = 2
end
const neigh_number = (2*nps + 1)^2 - 1 #Número de vecinos totales

const direc = pwd()*"/prueba_canonico_doctora/"

"/home/rafael/Documentos/Tesis/Programas/Tesis/prueba_canonico_doctora/"

In [2]:
const temperature = 0.1
const nsample = 50 #Pasos de Monte Carlo para tomar una muestra
const cycles = Int(fld(side^3.7, nsample)*nsample + nsample) #Para que sea múltiplo de nsample
#const cycles = floor(Int64, side^3.7)

9850

In [3]:
open(direc*"lambda"*"$(@sprintf("%.5f", lambda))"*"datos_simulacion.txt", "w") do data
    write(data, 
    "canonical mc\n",
    "side = $side\n","lambda = $lambda\n", "l1 = $l1\n", "l2 = $l2\n", "nps = $nps\n", 
    "neigh_number = $neigh_number\n", "temperature = $temperature\n", 
    "nsample = $nsample\n", "cycles = $cycles\n")
end

127

In [4]:
function square_well(r_ij::Vector{Float64})
    dist = norm(r_ij)
    if (dist == 0.0)
        ans = Inf
    elseif (dist > 0 && dist <= lambda)
        ans = -1.0
    elseif dist > lambda
        ans = 0.0
    end
    return ans
end

square_well (generic function with 1 method)

In [5]:
# Si x va hacia la derecha y y hacia arriba, este genera las partículas hacia arriba y luego hacia la derecha 
# (moléculas derechas)

# square_pos es la posición del cuadrado donde se situará la molécula
function dextro_molecule(square_pos::Vector{Float64})
    """
    Regresa una lista con las posiciones de los átomos de cada molécula para que sea derecha
    """
    pos = Vector{Vector}(3)
    pos[1] = [square_pos[1] - l2/2.0, square_pos[2] - l1/2.0]
    pos[2] = [square_pos[1] - l2/2.0, square_pos[2] + l1/2.0]
    pos[3] = [square_pos[1] + l2/2.0, square_pos[2] + l1/2.0]
    return pos
end

dextro_molecule (generic function with 1 method)

In [6]:
# Este genera las moléculas izquierdas
# Cuadrado donde se posicionará la molécula
function levo_molecule(square_pos::Vector{Float64})
    pos = Vector{Vector}(3)
    pos[1] = [square_pos[1] + l2/2.0, square_pos[2] - l1/2.0]
    pos[2] = [square_pos[1] + l2/2.0, square_pos[2] + l1/2.0]
    pos[3] = [square_pos[1] - l2/2.0, square_pos[2] + l1/2.0]
    return pos
end

levo_molecule (generic function with 1 method)

In [7]:
function create_molecules(nmol, ppm)
    mol_centers = Vector(nmol)
    positions = Vector(nmol)
    chirality = Vector(nmol)
    neighbors = Vector(nmol)
    
    # Hacer que haya la mitad de las moléculas tengan la misma quiralidad
    for ii in 1:convert(Int64, nmol/2)
        chirality[ii] = "L"
    end
    for ii in (convert(Int64, nmol/2) + 1):nmol
        chirality[ii] = "D"
    end
    #chir = transpose(reshape(chirality, (side, side)))
    #chirality = vec(chir)
    
    #shuffle!(chirality)
    # Otro método, pero no siempre serían exactamente la mitad de derechas e izquierdas
    # chirality = rand(["L", "D"], nsites)
        
    # Sitúa las moléculas en posiciones y orientaciones aleatorias
    for jj in 0:(side - 1)
        for ii in 0:(side - 1)
            mol_index = jj*side + ii + 1
            #mol_index = ii*side + jj + 1
            if chirality[mol_index] == "L"
                temp_pos = levo_molecule([0.0, 0.0])
            else
                temp_pos = dextro_molecule([0.0, 0.0])
            end
            mol_centers[mol_index] = [ii + 0.5, jj + 0.5] #Podría ser solamente [ii, jj]
            #En caso de rotación
            #orientation = rand(1:4)
            #positions[mol_index] = map(rot_int, temp_pos, fill(orientation, ppm)) + fill(mol_centers[mol_index], ppm)
            positions[mol_index] = temp_pos + fill(mol_centers[mol_index], ppm)
            
            # Si nps = 1, hay 8 vecinos. Si nps = 2, hay 24 vecinos.
            neighbors[mol_index] = [mod(jj + col, side)*side + mod(ii + row, side) + 1 
                for col in -nps:nps for row in -nps:nps]
            deleteat!(neighbors[mol_index], convert(Int64, neigh_number/2 + 1)) # Es la molécula central
        end
    end
    
    return mol_centers, positions, chirality, neighbors
end

create_molecules (generic function with 1 method)

In [8]:
# Esta función podría ir en dos partes: cambiar quiralidad y cambiar orientación
function change_molecule(mol_chirality::String, pos::Vector{Vector{Float64}}, center::Vector{Float64})

    if mol_chirality == "L"
        new_pos = dextro_molecule([0.0, 0.0]) + fill(center, atoms_per_molecule)
        new_mol_chirality = "D"
    else
        new_pos = levo_molecule([0.0, 0.0]) + fill(center, atoms_per_molecule)
        new_mol_chirality = "L"
    end

    # El método para seleccionar la rotación puede cambiar, si es que no es simétrico
    #orientation = rand(1:4)
    #new_pos = map(rot_int, temp_pos, fill(orientation, atoms_per_molecule)) + fill(center, atoms_per_molecule)
    #new_pos = temp_pos + fill(center, atoms_per_molecule)
    return new_mol_chirality, new_pos
end

change_molecule (generic function with 1 method)

In [9]:
function potential_energy(positions::Vector)
    # positions incluye las posiciones de cada átomo
    # positions[1] es la molécula central
    energy = 0.0
    for kk in 2:(neigh_number + 1)
        for jj in 1:3
            for ii in 1:3
                # Sin condiciones periódicas
                #energy += square_well(positions[1][ii] - positions[kk][jj])
                
                #Con condiciones periódicas
                dist = Vector{Float64}(2)
                for aa in 1:2    
                    temp_dist = abs(positions[1][ii][aa] - positions[kk][jj][aa])
                    if temp_dist > side/2
                        temp_dist = side - temp_dist
                    end
                    dist[aa] = temp_dist
                end
                #println(dist)
                # Otra manera de hacer lo de arriba (la de arriba ya está probada)
                #temp_dist = abs(positions[1][ii] - positions[kk][jj])
                #dist = temp_dist - side*round(temp_dist/side, RoundNearestTiesAway)
                energy += square_well(dist)
                #println(energy)
            end
        end
    end
    #println(energy)
    return energy
end

potential_energy (generic function with 1 method)

In [10]:
centers, positions, chirality, neighbors = create_molecules(floor(Int64, side*(side)), atoms_per_molecule)

(Any[[0.5,0.5],[1.5,0.5],[2.5,0.5],[3.5,0.5],[4.5,0.5],[5.5,0.5],[6.5,0.5],[7.5,0.5],[8.5,0.5],[9.5,0.5]  …  [2.5,11.5],[3.5,11.5],[4.5,11.5],[5.5,11.5],[6.5,11.5],[7.5,11.5],[8.5,11.5],[9.5,11.5],[10.5,11.5],[11.5,11.5]],Any[Array{Float64,1}[[0.725,0.05],[0.725,0.95],[0.275,0.95]],Array{Float64,1}[[1.725,0.05],[1.725,0.95],[1.275,0.95]],Array{Float64,1}[[2.725,0.05],[2.725,0.95],[2.275,0.95]],Array{Float64,1}[[3.725,0.05],[3.725,0.95],[3.275,0.95]],Array{Float64,1}[[4.725,0.05],[4.725,0.95],[4.275,0.95]],Array{Float64,1}[[5.725,0.05],[5.725,0.95],[5.275,0.95]],Array{Float64,1}[[6.725,0.05],[6.725,0.95],[6.275,0.95]],Array{Float64,1}[[7.725,0.05],[7.725,0.95],[7.275,0.95]],Array{Float64,1}[[8.725,0.05],[8.725,0.95],[8.275,0.95]],Array{Float64,1}[[9.725,0.05],[9.725,0.95],[9.275,0.95]]  …  Array{Float64,1}[[2.275,11.05],[2.275,11.95],[2.725,11.95]],Array{Float64,1}[[3.275,11.05],[3.275,11.95],[3.725,11.95]],Array{Float64,1}[[4.275,11.05],[4.275,11.95],[4.725,11.95]],Array{Float64,1}[[5.

In [11]:
reshape(chirality, (side, side))

12×12 Array{Any,2}:
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"

In [12]:
energy_mat = Vector{Int64}(side*side)
for idx in 1:(side*side)
    #idx = jj*side + ii + 1
    neighbor_pos = Vector(neigh_number + 1)
    for (ii, molecules) in enumerate([idx, neighbors[idx]...])
        neighbor_pos[ii] = positions[molecules]
    end
    energy_mat[idx] = potential_energy(neighbor_pos)
end 
reshape(energy_mat, (side, side))

12×12 Array{Int64,2}:
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19

In [20]:
change_idx = 65
chirality[idx], positions[idx] = change_molecule(chirality[change_idx], positions[change_idx], centers[change_idx])
reshape(chirality, (side, side))

12×12 Array{Any,2}:
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "D"
 "L"  "L"  "L"  "L"  "L"  "L"  "D"  "D"  "D"  "D"  "D"  "L"

In [21]:
energy_mat2 = Vector{Int64}(side*side)
for idx in 1:(side*side)
    #idx = jj*side + ii + 1
    neighbor_pos = Vector(neigh_number + 1)
    for (ii, molecules) in enumerate([idx, neighbors[idx]...])
        neighbor_pos[ii] = positions[molecules]
    end
    energy_mat2[change_idx] = potential_energy(neighbor_pos)
end 
reshape(energy_mat, (side, side))

12×12 Array{Int64,2}:
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -21  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -19  -18  -20  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -19  -18  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19
 -19  -20  -20  -20  -20  -19  -19  -20  -20  -20  -20  -19

In [16]:
function metropolis_step(exp_arg::Float64)
    """
    Returns True if the trial configuration goes towards a region with higher probability
    or it is given the chance to explore regions with fewer probability
    """
    if exp_arg < 0.0
        return true # Se actualiza el estado del sistema
    else
        w = exp(-exp_arg)
        if rand() < w # Energía: w
            return true # También se actualiza el estado del sistema
        else
            return false
        end
    end
end

metropolis_step (generic function with 1 method)

In [22]:
metropolis_step(Float64(energy_mat2[change_idx] - energy_mat[change_idx]))

false

In [53]:
function montecarlo!(centers::Vector, positions::Vector, chirality::Vector, neighbors::Vector, t_env)
    """
    Tries to perform a change in the system, like changing a spin direction
    """
    
    # Elegir una molécula al azar
    idx = rand(1:(side*side))
        
    # Buscar las posiciones de los átomos de esa molécula y de sus vecinas
    neighbor_pos = Vector(neigh_number + 1)
    for (ii, molecules) in enumerate([idx, neighbors[idx]...])
        neighbor_pos[ii] = positions[molecules]
    end
    #println("Old positions: ", neighbor_pos[1])
    #println("Old chirality: ", chirality[idx])
    
    # Intento de cambio
    trial_chir, trial_pos = change_molecule(chirality[idx], positions[idx], centers[idx])
    neighbor_pos2 = Vector(neigh_number + 1)
    neighbor_pos2[1] = trial_pos
    for (ii, molecules) in enumerate(neighbors[idx])
        neighbor_pos2[ii + 1] = positions[molecules]
    end
    #println("Trial positions: ", neighbor_pos2[1])
    #println("Trial chirality: ", trial_chir)
    

    # Cálculo de energías
    old_energy = potential_energy(neighbor_pos)
    trial_energy = potential_energy(neighbor_pos2)
    #println("Old E: ", old_energy)
    #println("Trial E: ", trial_energy)

    # La diferencia de energías se utiliza para aceptar o rechazar el movimiento. 
    # energy_change entrega la diferencia de energías si el movimiento fue aceptado.
    # Si fue rechazado, no hay cambio de energía en el sistema.
    energy_diff = trial_energy - old_energy
    energy_change = 0.0
    if metropolis_step(energy_diff/t_env)
        positions[idx] = trial_pos
        chirality[idx] = trial_chir
        energy_change = energy_diff
    end
    #println("New positions: ", positions[idx])
    #println("New chirality: ", chirality[idx])
    
    return positions, chirality, energy_change
end

montecarlo! (generic function with 1 method)

In [54]:
function montecarlo_cycle!(centers, positions, chirality, neighbors, t_env, ncycles)
    n_levo = []
    n_dextro = []
    energies = []
    
    # Calcular energía total del sistema al inicio. Después se sumará (o restará)
    # la diferencia de energías de cada paso de Montecarlo, para no tener qué 
    # calcular la energía total a cada rato
    te = 0.0
    for jj in 1:(side*side)
        neighbor_pos = Vector(neigh_number + 1)
        for (ii, molecules) in enumerate([jj, neighbors[jj]...])
            neighbor_pos[ii] = positions[molecules]
        end
        te += potential_energy(neighbor_pos)
    end
    # el /2 es porque se cuentan los pares dos veces
    te = te/2.0
    #println(te)
    dextro_counter = count(kk -> kk == "D", chirality)
    levo_counter = count(kk -> kk == "L", chirality)
    append!(n_dextro, dextro_counter)
    append!(n_levo, levo_counter)
    append!(energies, te/(side*side))
    #println("Energía inicial: ", te)

    for kk in 1:ncycles
        positions, chirality, energy_change = montecarlo!(centers, positions, chirality, neighbors, t_env)
        te += energy_change
        #println(te)
        # Calcular cosas
        if mod(kk, nsample) == 0
            # Con estos también podría hacerse lo mismo de contar moléculas con cierta quiralidad desde antes
            dextro_counter = count(kk -> kk == "D", chirality)
            levo_counter = count(kk -> kk == "L", chirality)
            append!(n_dextro, dextro_counter)
            append!(n_levo, levo_counter)
            
            append!(energies, te/(side*side))
        end
    end
    return n_levo, n_dextro, energies
end

montecarlo_cycle! (generic function with 1 method)

In [55]:
function repeat_cycles(reps)
    
    levos_per_rep = Vector(reps)
    dextros_per_rep = Vector(reps)
    energies_per_rep = Vector(reps)
    
    for jj in 1:reps
        centers, positions, chirality, neighbors = create_molecules(floor(Int64, side*(side)), atoms_per_molecule)
        q1 = deepcopy(chirality)
        levos, dextros, energies = montecarlo_cycle!(centers, positions, chirality, neighbors, temperature, cycles)
        levos_per_rep[jj] = levos
        dextros_per_rep[jj] = dextros
        energies_per_rep[jj] = energies
        
        q1n = Vector(side*side)
        q2n = Vector(side*side)
        for ii in 1:(side*side)
            if q1[ii] == "D"
                q1n[ii] = 1
            else
                q1n[ii] = 0
            end
            if chirality[ii] == "D"
                q2n[ii] = 1
            else
                q2n[ii] = 0
            end
        end
        writedlm(direc*"lambda"*string(lambda)*"_"*string(jj)*"_inicial.txt", reshape(q1n, (side, side)))
        writedlm(direc*"lambda"*string(lambda)*"_"*string(jj)*"_final.txt", reshape(q2n, (side, side)))
    end
    return levos_per_rep, dextros_per_rep, energies_per_rep
end

repeat_cycles (generic function with 1 method)

In [56]:
@time t_levos, t_dextros, t_energies = repeat_cycles(5)

 87.366616 seconds (565.44 M allocations: 12.652 GB, 10.66% gc time)


(Any[Any[200,188,191,198,197,202,207,208,213,209  …  400,400,400,400,400,400,400,400,400,400],Any[200,204,199,198,197,201,200,201,198,199  …  200,200,200,200,200,200,200,200,200,200],Any[200,204,206,206,204,205,206,204,211,204  …  400,400,400,400,400,400,400,400,400,400],Any[200,195,201,205,201,204,201,199,204,205  …  280,280,280,280,280,280,280,280,280,280],Any[200,202,201,196,204,200,203,198,195,188  …  0,0,0,0,0,0,0,0,0,0]],Any[Any[200,212,209,202,203,198,193,192,187,191  …  0,0,0,0,0,0,0,0,0,0],Any[200,196,201,202,203,199,200,199,202,201  …  200,200,200,200,200,200,200,200,200,200],Any[200,196,194,194,196,195,194,196,189,196  …  0,0,0,0,0,0,0,0,0,0],Any[200,205,199,195,199,196,199,201,196,195  …  120,120,120,120,120,120,120,120,120,120],Any[200,198,199,204,196,200,197,202,205,212  …  400,400,400,400,400,400,400,400,400,400]],Any[Any[-9.2475,-9.3,-9.3475,-9.3975,-9.4375,-9.495,-9.51,-9.53,-9.555,-9.58  …  -10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0],Any[-9.2825,-9.32

In [57]:
function write_data(data, file)
    rdata = Array{Float64}(length(data), length(data[1]))
    for jj in 1:length(data) #4
        for ii in 1:length(data[1])
            rdata[jj, ii] = data[jj][ii]
        end
    end
    writedlm(direc*file, rdata)
end

write_data (generic function with 1 method)

In [58]:
write_data(t_dextros, "t_dextros_lambda"*string(lambda)*".txt")

In [59]:
write_data(t_energies, "t_energies_lambda"*string(lambda)*".txt")

In [60]:
write_data(t_levos, "t_levos_lambda"*string(lambda)*".txt")